# 記事

## 作るべきテーブルイメージ


補助金受領回数2020年度宮城県企業テーブルのイメージ：

| 企業ID | 企業名  | 所在地 | 補助金受領回数 |
|--------|--------|--------|----------------|
| 3      | 企業C  | 宮城県 | 3              |
| 1      | 企業A  | 宮城県 | 2              |
| 2      | 企業B  | 宮城県 | 1              |
| 4      | 企業D  | 宮城県 | 1              |


## 関係するAPIをトライ

In [169]:
# APIの基本設定
API_TOKEN = "YOUR_API_TOKEN"  # 実際のAPIトークンに置き換えてください
API_TOKEN = "Hgt7Gj3Il3rQovE8OiU817jgmrKe9jnM"  # テスト後に削除
BASE_URL = "https://info.gbiz.go.jp/hojin/v1/hojin"
HEADERS={
    'accept': 'application/json',
    'X-hojinInfo-api-token': API_TOKEN
}

In [20]:
import requests
from pprint import pprint

response = requests.get(
    BASE_URL, headers=HEADERS,
    params={
        'prefecture': '04',
        'year': '2020',
        'source': '5',
        'page': '1',
        'limit': '1'
    }
)

pprint(response.json())

{'errors': None,
 'hojin-infos': [{'corporate_number': '1000020042129',
                  'location': '宮城県登米市迫町佐沼字中江２丁目６－１',
                  'name': '登米市',
                  'name_en': 'Tome City',
                  'number_of_activity': '207',
                  'postal_code': '9870511',
                  'status': '-',
                  'update_date': '2018-04-02T00:00:00+09:00'}],
 'id': None,
 'message': '200 - OK.'}


In [25]:
response = requests.get(
  f'{BASE_URL}/1000020042129/subsidy', headers=HEADERS,
)
pprint(response.json())

{'errors': None,
 'hojin-infos': [{'corporate_number': '1000020042129',
                  'kana': 'とめし',
                  'location': '宮城県登米市迫町佐沼字中江２丁目６－１',
                  'name': '登米市',
                  'name_en': 'Tome City',
                  'postal_code': '9870511',
                  'status': '-',
                  'subsidy': [{'amount': None,
                               'date_of_approval': None,
                               'government_departments': '経済産業省',
                               'joint_signatures': None,
                               'note': None,
                               'subsidy_resource': None,
                               'target': None,
                               'title': '令和２年度エネルギー構造高度化・転換理解促進事業費補助金（登米市\u3000'
                                        '地域資源を活用した再生可能エネルギー設備導入事業）'},
                              {'amount': '25704000',
                               'date_of_approval': '2024-08-27',
                               'government_de

## DB 設計

In [170]:
from sqlalchemy import create_engine, Column, String, Integer, Date, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base


# データベースエンジンの作成（SQLiteを使用）
DATABASE_URL = 'sqlite:///gbizinfo_miyagi_2020.sqlite3'
engine = create_engine(DATABASE_URL, echo=True)

# Baseクラスの作成
Base = declarative_base()

# 2020年度宮城県企業マスターテーブルの定義
class MiyagiCompanies2020(Base):
    __tablename__ = 'companies'
    
    corporate_number = Column(String, primary_key=True)  # 法人番号（13桁）
    name = Column(String)  # 法人名
    location = Column(String)  # 本社所在地

class MiyagiSubsidies2020(Base):
    __tablename__ = 'subsidies'

    id = Column(Integer, primary_key=True, autoincrement=True)
    corporate_number = Column(String, ForeignKey('companies.corporate_number'))  # 企業の法人番号
    subsidy_title = Column(String)  # 補助金タイトル
    amount = Column(Integer)  # 補助金額
    date_of_approval = Column(Date)  # 補助金承認日
    government_department = Column(String)  # 補助金の政府機関
    target = Column(String)  # 補助金の対象

    # 関連を定義（企業情報と補助金情報を関連付ける）
    company = relationship('MiyagiCompanies2020', backref='subsidies')




# 2020年度宮城県企業マスターテーブルの作成
Base.metadata.create_all(engine)


2025-04-02 16:01:14,649 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-02 16:01:14,651 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("companies")


2025-04-02 16:01:14,651 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 16:01:14,670 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("companies")
2025-04-02 16:01:14,671 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 16:01:14,672 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("subsidies")
2025-04-02 16:01:14,673 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 16:01:14,673 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("subsidies")
2025-04-02 16:01:14,675 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-04-02 16:01:14,681 INFO sqlalchemy.engine.Engine 
CREATE TABLE companies (
	corporate_number VARCHAR NOT NULL, 
	name VARCHAR, 
	location VARCHAR, 
	PRIMARY KEY (corporate_number)
)


2025-04-02 16:01:14,683 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2025-04-02 16:01:14,689 INFO sqlalchemy.engine.Engine 
CREATE TABLE subsidies (
	id INTEGER NOT NULL, 
	corporate_number VARCHAR, 
	subsidy_title VARCHAR, 
	amount INTEGER, 
	date_of_approval DATE, 

/var/folders/pv/3g2620ld3rs7s8h3vfphb4w80000gn/T/ipykernel_2236/126610107.py:11: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## データ取得

In [171]:
import requests
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# データベース設定
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


# データ取得＆DB保存
for page in range(1, 11):  # page=1 から page=10 まで
    response = requests.get(
        BASE_URL, headers=HEADERS,
        params={
            'prefecture': '04',  # 宮城県
            'year': '2020',
            'source': '5',
            'page': page,
            'limit': 5000
        }
    )
    
    if response.status_code != 200:
        print(f"エラー: {response.status_code}")
        continue
    
    data = response.json()
    
    for company in data.get('hojin-infos', []):
        new_company = MiyagiCompanies2020(
            corporate_number=company.get('corporate_number'),
            name=company.get('name'),
            location=company.get('location')
        )
        session.add(new_company)

    session.commit()  # 取得したデータをコミット
    print(f"Page {page} のデータを保存しました")

print("データ取得完了！")

/var/folders/pv/3g2620ld3rs7s8h3vfphb4w80000gn/T/ipykernel_2236/2738009454.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


2025-04-02 16:01:19,172 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-02 16:01:19,177 INFO sqlalchemy.engine.Engine INSERT INTO companies (corporate_number, name, location) VALUES (?, ?, ?)
2025-04-02 16:01:19,178 INFO sqlalchemy.engine.Engine [generated in 0.00159s] [('1000020042129', '登米市', '宮城県登米市迫町佐沼字中江２丁目６－１'), ('1000020042137', '栗原市', '宮城県栗原市築館薬師１丁目７番１号'), ('1000020042153', '大崎市', '宮城県大崎市古川七日町１－１'), ('1000020044067', '利府町', '宮城県宮城郡利府町利府字新並松４'), ('1010001173009', '株式会社ジーデップ・アドバンス', '宮城県仙台市青葉区国分町３丁目４番３３'), ('1030001061582', '株式会社ＭＳＣ', '宮城県仙台市太白区鹿野１丁目４番９'), ('1370001000886', '株式会社アイリスプラザ', '宮城県仙台市青葉区中央２丁目１番７号'), ('1370001001067', 'パイオニアシステムテクノロジー株式会社', '宮城県仙台市青葉区堤町１丁目１番２号')  ... displaying 10 of 356 total bound parameter sets ...  ('9370802000012', '有限会社さくら', '宮城県亘理郡亘理町長瀞字大橋２６１番地４'), ('9400001004769', '株式会社ジェイウッド', '宮城県仙台市泉区大沢３丁目４番１')]
2025-04-02 16:01:19,181 INFO sqlalchemy.engine.Engine COMMIT
Page 1 のデータを保存しました
エラー: 404
エラー: 404
エラー: 404
エラー: 404
エラー: 404
エラー: 404
エラー: 4

In [61]:
# 2020に補助金を受けた宮城県の企業
print(len(response.json().get("hojin-infos", [])))


1


In [ ]:
import requests
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

# データベース接続
DATABASE_URL = 'sqlite:///gbizinfo_miyagi_2020.sqlite3'
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()


# 企業リストを取得
companies = session.query(MiyagiCompanies2020).all()

for company in companies:
    corporate_number = company.corporate_number
    response = requests.get(f"{BASE_URL}/{corporate_number}/subsidy", headers=HEADERS)
    
    if response.status_code == 200:
        data = response.json()
        hojin_infos = data.get("hojin-infos", [])
        
        if hojin_infos:
            subsidies = hojin_infos[0].get("subsidy", [])
            
            for sub in subsidies:
                date_of_approval = sub.get("date_of_approval")
                
                if date_of_approval:
                    # 2020年度（2020年4月1日〜2021年3月31日）のみ抽出
                    date_of_approval = datetime.strptime(date_of_approval, '%Y-%m-%d').date()
                    fiscal_start = datetime(2020, 4, 1).date()
                    fiscal_end = datetime(2021, 3, 31).date()
                    
                    if fiscal_start <= date_of_approval <= fiscal_end:
                        subsidy_record = MiyagiSubsidies2020(
                            corporate_number=corporate_number,
                            subsidy_title=sub.get("title"),
                            amount=int(sub.get("amount") or 0),
                            date_of_approval=date_of_approval,
                            government_department=sub.get("government_departments"),
                            target=sub.get("target")
                        )
                        session.add(subsidy_record)
                    
    else:
        print(f"Error fetching data for {corporate_number}: {response.status_code}")

session.commit()
session.close()
print("補助金データの取得＆保存完了！")


補助金データの取得＆保存完了！


## データ操作

In [173]:
import pandas as pd

def query_to_dataframe(model):
    """SQLAlchemyのモデルをDataFrameに変換する"""
    df = pd.DataFrame([record.__dict__ for record in session.query(model).all()])
    df.drop(columns=['_sa_instance_state'], errors='ignore', inplace=True)
    return df

# 各テーブルのデータを取得してDataFrame化
df_subsidies = query_to_dataframe(MiyagiSubsidies2020)
df_companies = query_to_dataframe(MiyagiCompanies2020)



In [174]:
df_companies.head()

,name,corporate_number,location
0,登米市,1000020042129,宮城県登米市迫町佐沼字中江２丁目６－１
1,栗原市,1000020042137,宮城県栗原市築館薬師１丁目７番１号
2,大崎市,1000020042153,宮城県大崎市古川七日町１－１
3,利府町,1000020044067,宮城県宮城郡利府町利府字新並松４
4,株式会社ジーデップ・アドバンス,1010001173009,宮城県仙台市青葉区国分町３丁目４番３３


In [175]:
df_subsidies.head()

,amount,date_of_approval,government_department,corporate_number,id,subsidy_title,target
0,707000,2021-03-22,国土交通省,1000020042129,1,集約都市形成支援事業,None
1,7892000,2021-03-04,環境省,1000020042129,2,災害等廃棄物処理事業費補助金,None
2,170000,2021-02-24,厚生労働省,1000020042129,3,令和２年度新型コロナウイルス感染症対応医療機関労災給付上乗せ補償保険加入支援事業補助金,None
3,20801000,2021-02-17,国土交通省,1000020042129,4,河川等災害復旧事業,None
4,27927000,2020-09-24,環境省,1000020042129,5,循環型社会形成推進交付金,None


In [176]:
df_merge = df_subsidies.merge(df_companies, on='corporate_number', how='left')

中間データセット

In [206]:
df_subsidies_miyagi_2020 = df_merge[['corporate_number','name','subsidy_title','amount','date_of_approval']]
df_subsidies_miyagi_2020 = df_subsidies_miyagi_2020.sort_values('corporate_number')
df_subsidies_miyagi_2020.to_csv("subsidies_miyagi_2020.csv")
df_subsidies_miyagi_2020.head()

,corporate_number,name,subsidy_title,amount,date_of_approval
0,1000020042129,登米市,集約都市形成支援事業,707000,2021-03-22
1,1000020042129,登米市,災害等廃棄物処理事業費補助金,7892000,2021-03-04
2,1000020042129,登米市,令和２年度新型コロナウイルス感染症対応医療機関労災給付上乗せ補償保険加入支援事業補助金,170000,2021-02-24
3,1000020042129,登米市,河川等災害復旧事業,20801000,2021-02-17
4,1000020042129,登米市,循環型社会形成推進交付金,27927000,2020-09-24


In [204]:
df_subsidies_count = df_subsidies_miyagi_2020.groupby('corporate_number').agg(
    name=('name','first'),
    id=('name', 'count'), 
    total_amount=('amount', 'sum'),  # 合計金額を集計
).reset_index().rename(columns={'id': 'subsidy_count', 'total_amount':'total_amount'})

df_subsidies_count = df_subsidies_count.sort_values('subsidy_count', ascending=False)
df_subsidies_count.head()


,corporate_number,name,subsidy_count,total_amount
48,8000020040002,宮城県,271,141867868458
49,8000020041009,仙台市,47,13951446964
47,7370005002147,国立大学法人東北大学,46,4408187784
50,8000020042056,気仙沼市,21,5534688685
56,8370005002146,国立大学法人宮城教育大学,19,152132700
